# Production

## Import

In [ ]:
import tensorflow as tf # type: ignore
# tf.compat.v1.disable_v2_behavior()
# import tensorflow_decision_forests as tfdf
import pandas as pd # type: ignore
import numpy as np
from numpy.random import seed
from sklearn.model_selection import (train_test_split, cross_val_score,
                                     KFold,ShuffleSplit)
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
from keras import backend as K
from IPython.display import clear_output

import pickle
import time, datetime
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
# K.set_session(sess)

# tf.config.list_logical_devices('GPU')
grand_start=time.time()
SEED=7
tf.random.set_seed(SEED)

seed(SEED)
random_state=SEED
from tensorflow.keras import initializers
from tqdm import tqdm

## Load Data

In [ ]:
CASE_DF2=pd.read_feather(f'/path/to/CASE_DF2.feather')
CONTROL_DF2=pd.read_feather(f'/path/to/CONTROL_DF2.feather')

In [ ]:
CASE_DF2.columns

In [ ]:
CONTROL_DF2.columns

## SETUP

### n_1, n_0

In [ ]:
n_1=CASE_DF2.stay_id.nunique()
n_0=CONTROL_DF2.stay_id.nunique()

In [ ]:
n_1,n_0

### Train test split

In [ ]:
TS=pd.concat([CASE_DF2,CONTROL_DF2],axis=0)

X=np.concatenate((CASE_DF2.stay_id.unique(), CONTROL_DF2.stay_id.unique()))
y=[int(i) for i in np.concatenate((np.ones(n_1),np.zeros(n_0)))]

X_train_all, X_test, y_train_all, y_test=train_test_split(X,y                                                  
                                                  ,test_size=0.3
                                                  ,random_state=random_state
                                                  ,shuffle=True
                                                 )
X_train, X_val, y_train, y_val=train_test_split(X_train_all,y_train_all
                                                  ,test_size=0.3
                                                  ,random_state=random_state
                                                  ,shuffle=True
                                                 )


#### DATA เอาไว้ compare

In [ ]:
DATA=pd.DataFrame(np.vstack((X,np.array(y))).transpose(),columns=['stay_id','y'])

## Data Preparation

In [ ]:
ts_columns=[
       'HR',
       'Non Invasive Blood Pressure systolic',
       'Non Invasive Blood Pressure mean', 'RR', 'Arterial O2 pressure',
       'Arterial CO2 Pressure', 'O2 saturation pulseoxymetry',
       'Hematocrit (serum)', 'WBC', 'Creatinine (serum)', 'Glucose (serum)',
       'Sodium (serum)', 'TemperatureF', 'PH (Arterial)',
       'Inspired O2 Fraction', 'BUN', 'Total Bilirubin',
       'Potassium (serum)', 'HCO3 (serum)', 'Albumin', 'UrineVolumeOut',
       'GCS',
       ]

static_columns=[
       'anchor_age',
       'G_01',
       'G_02', 'G_03', 'G_04', 'G_05', 'G_06', 'G_07', 'G_08', 'G_09', 'G_10',
       'G_11', 'G_12', 'G_13', 'G_14', 'G_15', 'G_17', 'G_18', 'G_19', 'G_20',
       'G_21', 'count_diags'
]

NEWS_columns=[
        'HR','TemperatureF','RR','Non Invasive Blood Pressure systolic','O2 saturation pulseoxymetry',
]

OASIS_columns=[
        'HR','TemperatureF','GCS','UrineVolumeOut','RR','Non Invasive Blood Pressure mean'
]

SAP_columns=[
        'HR','TemperatureF','GCS','UrineVolumeOut','Non Invasive Blood Pressure systolic',
        'BUN','Inspired O2 Fraction','HCO3 (serum)','Arterial O2 pressure',
        'Potassium (serum)','Sodium (serum)','Total Bilirubin','WBC',
]

APACHE_columns=[
        'HR','TemperatureF','GCS','UrineVolumeOut','RR','Non Invasive Blood Pressure mean',
        'Albumin','BUN','Creatinine (serum)','Inspired O2 Fraction','Glucose (serum)','Hematocrit (serum)',        
        'Arterial CO2 Pressure','PH (Arterial)','Arterial O2 pressure',
        'Sodium (serum)','Total Bilirubin','WBC',
]

ALLcommon_columns=[
        'HR','TemperatureF','GCS','UrineVolumeOut','RR',
]

ALLunion_columns=ts_columns

#### เอา 'Total Bilirubin','Arterial O2 pressure','BUN', ออก --> AUC เหลือ 0.85


# static_columns=[
#        'anchor_age','count_diags'
# ]

_ts_columns=ts_columns
# _ts_columns=NEWS_columns
_static_columns=static_columns

# Explore

In [ ]:
"""
TS/STATIC
"""
#### TRAIN ALL ####
_x_train_all_ts=None
_y_train_all=None
for i,g in TS[TS['stay_id'].isin(X_train_all)].groupby('stay_id'):
    _g=g.sort_values('hourfromouttime',ascending=True)[_ts_columns].values
    if _x_train_all_ts is None:
        _x_train_all_ts=[[i,_g]]
    else:
        _x_train_all_ts=np.append(_x_train_all_ts,[[i,_g]], axis=0)
    
    _tmp_y=DATA.query(f'stay_id=={i}').y.values[0].astype(int)
    if _y_train_all is None:
        _y_train_all=[[i,_tmp_y]]
    else:
        _y_train_all=np.append(_y_train_all,[[i,_tmp_y]], axis=0)

print('_x_train_all_ts')


#### TRAIN ####
_x_train_ts=None
_y_train=None
for i,g in TS[TS['stay_id'].isin(X_train)].groupby('stay_id'):
    _g=g.sort_values('hourfromouttime',ascending=True)[_ts_columns].values
    if _x_train_ts is None:
        _x_train_ts=[[i,_g]]
    else:
        _x_train_ts=np.append(_x_train_ts,[[i,_g]], axis=0)

    _tmp_y=DATA.query(f'stay_id=={i}').y.values[0].astype(int)
    if _y_train is None:
        _y_train=[[i,_tmp_y]]
    else:
        _y_train=np.append(_y_train,[[i,_tmp_y]], axis=0)
print('_x_train_ts')

#### VALIDATE ####
_x_val_ts=None
_y_val=None
for i,g in TS[TS['stay_id'].isin(X_val)].groupby('stay_id'):
    _g=g.sort_values('hourfromouttime',ascending=True)[_ts_columns].values
    if _x_val_ts is None:
        _x_val_ts=[[i,_g]]
    else:
        _x_val_ts=np.append(_x_val_ts,[[i,_g]], axis=0)

    _tmp_y=DATA.query(f'stay_id=={i}').y.values[0].astype(int)
    if _y_val is None:
        _y_val=[[i,_tmp_y]]
    else:
        _y_val=np.append(_y_val,[[i,_tmp_y]], axis=0)
        
print('_x_val_ts')

#### TEST ####
_x_predict_ts=None
_y_predict=None
for i,g in TS[TS['stay_id'].isin(X_test)].groupby('stay_id'):
    _g=g.sort_values('hourfromouttime',ascending=True)[_ts_columns].values
    if _x_predict_ts is None:
        _x_predict_ts=[[i,_g]]
    else:
        _x_predict_ts=np.append(_x_predict_ts,[[i,_g]], axis=0)
        
    _tmp_y=DATA.query(f'stay_id=={i}').y.values[0].astype(int)
    if _y_predict is None:
        _y_predict=[[i,_tmp_y]]
    else:
        _y_predict=np.append(_y_predict,[[i,_tmp_y]], axis=0)
print('_x_predict_ts')

#### CONSTANT ####
n_ts_columns=len(_ts_columns)
n_static_columns=len(_static_columns)
print(f"n_ts_columns:{n_ts_columns}, n_static_columns:{n_static_columns}")

In [ ]:
_x_train_all_ts_df=pd.DataFrame(_x_train_all_ts,columns=['stay_id','data'])
_x_train_ts_df=pd.DataFrame(_x_train_ts,columns=['stay_id','data'])
_x_val_ts_df=pd.DataFrame(_x_val_ts,columns=['stay_id','data'])
_x_predict_ts_df=pd.DataFrame(_x_predict_ts,columns=['stay_id','data'])


In [ ]:
_y_train_all_df=pd.DataFrame(_y_train_all,columns=['stay_id','y'])
_y_train_df=pd.DataFrame(_y_train,columns=['stay_id','y'])
_y_val_df=pd.DataFrame(_y_val,columns=['stay_id','y'])
_y_predict_df=pd.DataFrame(_y_predict,columns=['stay_id','y'])

### Scale

In [ ]:
_n=np.array([i[1] for i in _x_train_all_ts])

In [ ]:
scaler_ts = StandardScaler()
scaler_ts.fit(_n.reshape(_n.shape[0]*_n.shape[1],_n.shape[2]))

In [ ]:
# Save StandardScaler
f=open('_model/scaler_ts.pickle','wb')
pickle.dump(scaler_ts,f)
f.close()

### Scale DF

In [ ]:
_x_train_all_ts_df['data_scale']=None
_x_train_ts_df['data_scale']=None
_x_val_ts_df['data_scale']=None
_x_predict_ts_df['data_scale']=None

for i,r  in _x_train_all_ts_df.iterrows():
    r['data_scale']=scaler_ts.transform(r['data'])
for i,r  in _x_train_ts_df.iterrows():
    r['data_scale']=scaler_ts.transform(r['data'])
for i,r  in _x_val_ts_df.iterrows():
    r['data_scale']=scaler_ts.transform(r['data'])
for i,r  in _x_predict_ts_df.iterrows():
    r['data_scale']=scaler_ts.transform(r['data'])


## Save Train_all/train/val/predict

## Hyper Parameters

In [ ]:
MODEL='PROD'
SERIAL='003'
epochs=500

In [ ]:
hyper_params={}
models=['dense','rnn','conv1d','lstm','gru']
feature_set=['NEWS','OASIS','SAP','APACHE','ALLcommon','ALLunion']
observations=[72,48,24,12,6]

feature_set_cols={}
feature_set_cols['NEWS']=len(NEWS_columns)
feature_set_cols['OASIS']=len(OASIS_columns)
feature_set_cols['SAP']=len(SAP_columns)
feature_set_cols['APACHE']=len(APACHE_columns)
feature_set_cols['ALLcommon']=len(ALLcommon_columns)
feature_set_cols['ALLunion']=len(ALLunion_columns)

for i in models:
    for j in feature_set:
        for k in observations:
            hyper_params[f'{i}_{j}_{k}']={                
                'model_arch': f'{i}',
                'feature_set': f'{j}',
                'observation': k,
                'cols': feature_set_cols[j]
                
            }

## Evaluation

### Confusion Matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels


def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    print(f'len(y_true): {len(y_true)}, len(y_pred): {len(y_pred)}')
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    nm=""
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        nm = "Norm"
        print("Normalized confusion matrix")
    else:
        nm = "withoutNorm"
        print('Confusion matrix, without normalization')

    print(cm.round(2))
    cm=np.flip(np.flip(cm,-1),0)
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

### AUROC

In [ ]:
from sklearn import metrics
def auroc(model, _y_predict, y_hat):
    fpr, tpr, thresholds = metrics.roc_curve(_y_predict, y_hat)
    auc_keras=metrics.auc(fpr, tpr)
    acc_per_fold.append(auc_keras)
    print(f"auc_keras: {auc_keras:.4f}")
    # calculate the g-mean for each threshold
    gmeans = np.sqrt(tpr * (1-fpr))
    J=tpr-fpr
    # locate the index of the largest g-mean
    ix = np.argmax(gmeans)
    ixJ= np.argmax(J)
    plt.figure(1, figsize=(5,5))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label=f'{model.name} (area = {auc_keras:.4f})')
    plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Youden\'s index=%.2f' % (thresholds[ixJ]))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title(f'ROC curve - {folder} MIMIC-IV')
    plt.legend(loc='best')
    plt.show()
    plt.savefig(f'{MODEL}/{model.name}.png')
    now=datetime.datetime.now().strftime("%Y%m%d-%H%M%S")    
    logdata=f'{now},{model.name},{auc_keras:.4f}'

    print(f'thresholds: {thresholds[ixJ]:.4f}')
    y_p = [ 0 if i <=thresholds[ixJ] else 1 for i in y_hat ]
    # plot_confusion_matrix(y_test, y_p, classes=(0,1), normalize=True, title=f"{model.name}")
    plot_confusion_matrix(_y_predict, y_p, classes=(0,1), normalize=True, title=f"{model.name}")
    return auc_keras

## Other settings

In [ ]:
"""
Imbalance dataset
"""
neg, pos = np.bincount(y)
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))
initial_bias = np.log([pos/neg])

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
print(class_weight)
print(f"initial_bias: {initial_bias}")
print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

"""
เก็บค่า
"""
from IPython.display import clear_output
acc_per_fold=[]
loss_per_fold = []
model_fold=[]
score_fold=[]

"""
Metrics
"""
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'), 
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

"""
Kernel Initializer
"""
kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.01, seed=SEED)

## Models

### dense

In [ ]:
def model_dense(param,fold=0,debug=True):

    model_arch=param['model_arch']
    feature_set=param['feature_set']
    observation=param['observation']
    rows=observation
    cols=param['cols']

    inputs=tf.keras.layers.Input(shape=(rows,cols)
                                 , name='input')
    flattern1=tf.keras.layers.Flatten(name='flattern1')(inputs)    
    ts1=tf.keras.layers.Dense(units=(rows*cols)
                               , kernel_initializer=kernel_initializer
                               , name='dense_1')(flattern1)
    dropout1=tf.keras.layers.Dropout(0.8)(ts1)
    ts2=tf.keras.layers.Dense(units=(rows*cols)
                               , kernel_initializer=kernel_initializer
                               , name='dense_2')(dropout1)

    outputs=tf.keras.layers.Dense(1
                                  , kernel_initializer=kernel_initializer
                                  , activation='sigmoid'
                                  , name='output')(ts2)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model._name=f'{model_arch}_{feature_set}_{observation}_fold{fold}'

    model.compile(
        loss='binary_crossentropy'
        ,metrics=[METRICS]
    )
    
    if debug:
        model.summary()
    return model

### conv1d

In [ ]:
def model_conv1d(param,fold=0,debug=True):
    model_arch=param['model_arch']
    feature_set=param['feature_set']
    observation=param['observation']
    rows=observation
    cols=param['cols']

    inputs=tf.keras.layers.Input(shape=(rows,cols))

    ts1=tf.keras.layers.Conv1D(filters=1
                               , kernel_size=round(rows/2)
                               , kernel_initializer=kernel_initializer
                               , name='conv1d_1')(inputs)
    dropout1=tf.keras.layers.Dropout(0.8)(ts1)
    ts2=tf.keras.layers.Conv1D(filters=1
                               , kernel_size=round(rows/2)
                               , kernel_initializer=kernel_initializer
                               , name='conv1d_2')(dropout1)


    flattern1=tf.keras.layers.Flatten(name='flattern_1')(ts2)
    outputs=tf.keras.layers.Dense(1
                                  , activation='sigmoid'
                                  , kernel_initializer=kernel_initializer
                                  , name='output')(flattern1)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model._name=f'{model_arch}_{feature_set}_{observation}_fold{fold}'

    model.compile(
        loss='binary_crossentropy'
        ,metrics=[METRICS]
    )
    
    if debug:
        model.summary()
    return model

### rnn

In [ ]:
def model_rnn(param,fold=0,debug=True):
    model_arch=param['model_arch']
    feature_set=param['feature_set']
    observation=param['observation']
    rows=observation
    cols=param['cols']

    inputs=tf.keras.layers.Input(shape=(rows,cols))

    ts1=tf.keras.layers.SimpleRNN(
                                 units=round(rows/2)
                               , dropout=0.8
                               , return_sequences=True
                               , kernel_initializer=kernel_initializer
                               , name='rnn_1')(inputs)
    ts2=tf.keras.layers.SimpleRNN(
                                 units=round(rows/4)
                               , return_sequences=False
                               , kernel_initializer=kernel_initializer
                               , name='rnn_2')(ts1)
    outputs=tf.keras.layers.Dense(1
                                  , kernel_initializer=kernel_initializer
                                  , activation='sigmoid'
                                  , name='output')(ts2)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model._name=f'{model_arch}_{feature_set}_{observation}_fold{fold}'

    model.compile(
        loss='binary_crossentropy'
        ,metrics=[METRICS]
    )
    
    if debug:
        model.summary()
    return model

### lstm

In [ ]:
def model_lstm(param,fold=0,debug=True):
    model_arch=param['model_arch']
    feature_set=param['feature_set']
    observation=param['observation']
    rows=observation
    cols=param['cols']

    inputs=tf.keras.layers.Input(shape=(rows,cols))

    ts1=tf.keras.layers.LSTM(
                                 units=round(rows/2)
                               , dropout=0.8
                               , return_sequences=True
                               , kernel_initializer=kernel_initializer
                               , name='lstm_1')(inputs)
    ts2=tf.keras.layers.LSTM(
                                 units=round(rows/4)
                               , return_sequences=False
                               , kernel_initializer=kernel_initializer
                               , name='lstm_2')(ts1)

    outputs=tf.keras.layers.Dense(1
                                  , kernel_initializer=kernel_initializer
                                  , activation='sigmoid'
                                  , name='output')(ts2)
    

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model._name=f'{model_arch}_{feature_set}_{observation}_fold{fold}'

    model.compile(
        loss='binary_crossentropy'
        ,metrics=[METRICS]
    )
    
    if debug:
        model.summary()
    return model

### gru

In [ ]:
def model_gru(param,fold=0,debug=True):
    model_arch=param['model_arch']
    feature_set=param['feature_set']
    observation=param['observation']
    rows=observation
    cols=param['cols']

    inputs=tf.keras.layers.Input(shape=(rows,cols))

    ts1=tf.keras.layers.GRU(
                                 units=round(rows/2)
                               , dropout=0.8
                               , return_sequences=True
                               , kernel_initializer=kernel_initializer
                               , name='gru_1')(inputs)
    ts2=tf.keras.layers.GRU(
                                 units=round(rows/4)
                               , return_sequences=False
                                , kernel_initializer=kernel_initializer
                               , name='gru_2')(ts1)
    outputs=tf.keras.layers.Dense(1
                                  , kernel_initializer=kernel_initializer
                                  , activation='sigmoid'
                                  , name='output')(ts2)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model._name=f'{model_arch}_{feature_set}_{observation}_fold{fold}'

    model.compile(
        loss='binary_crossentropy'
        ,metrics=[METRICS]
    )
    
    if debug:
        model.summary()
    return model

## System testing

### def get_datascale(param, data_scale)

In [ ]:
def get_datascale(param, data_scale):
    result=np.vstack(
        
            [
                data_scale[0:param['observation'],j] # rows
                for j in [ts_columns.index(i) for i in eval(f"{param['feature_set']}_columns")] # cols
            ]
        
        ).transpose()
    return result

## Cross Validation

In [ ]:
CV_X = np.array([ i.tolist() for i in _x_train_ts_df.data_scale])
CV_Predict_X=np.array([ i.tolist() for i in _x_predict_ts_df.data_scale])
CV_y = np.array(_y_train_df.y.values.tolist())
CV_Predict_y=np.array(_y_predict_df.y.values)


num_folds=5
kf = KFold(n_splits=num_folds, shuffle=False)

scores_fold={}

n_fold=1
for train_index, test_index in kf.split(CV_X):
        clear_output(wait=True)
        CV_X_train, CV_X_test = CV_X[train_index], CV_X[test_index]
        CV_y_train, CV_y_test = CV_y[train_index], CV_y[test_index]
        for i in hyper_params:
            param=hyper_params[i]
            model=eval(f"model_{param['model_arch']}(param,fold={n_fold},debug=False)")
            logdir = os.path.join(f"{MODEL}", f"{model._name}")
            tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
            history=model.fit(                       
                x=np.array([ get_datascale(param,i).tolist() 
                     for i in CV_X_train]),            
                y=CV_y_train,
                batch_size=100,
                verbose=0,
                epochs=100,
                class_weight=class_weight,
                callbacks=[tensorboard_callback],                
                validation_data=[np.array([ get_datascale(param,i).tolist() 
                     for i in CV_X_test]),CV_y_test]
            )
            scores = model.evaluate(
                x=np.array([ get_datascale(param,i).tolist() 
                     for i in CV_Predict_X]), 
                y=CV_Predict_y,
                batch_size=100,
                verbose=0)
            scores_fold[model.name]=scores
            with open('scores_fold.pickle', 'wb') as handle:
                pickle.dump(scores_fold, handle, protocol=pickle.HIGHEST_PROTOCOL)
                handle.close
        n_fold=n_fold+1
summary_score=[]
for i in scores_fold:
    summary_score.append(scores_fold[i])

In [ ]:
summary_score=[]
for i in scores_fold:
    summary_score.append([i]+scores_fold[i])
summary_score_df=pd.DataFrame(summary_score, columns=['model_name']+model.metrics_names)
summary_score_df.auc.describe()
summary_score_df.sort_values('auc', ascending=False)[0:5]

In [ ]:
import re
summary_score_df['observation']=summary_score_df['model_name']\
    .apply(lambda x: re.findall(r'_(\d{1,2})_fold',x)[0])
summary_score_df['model_architecture']=summary_score_df['model_name']\
    .apply(lambda x: re.findall(r'^(.*)_(.*)_(\d{1,2})_fold',x)[0][0])
summary_score_df['feature_set']=summary_score_df['model_name']\
    .apply(lambda x: re.findall(r'^(.*)_(.*)_(\d{1,2})_fold',x)[0][1])
summary_score_df.groupby('observation')['auc'].mean()\
    .reset_index(name='auc_mean')\
    .sort_values('auc_mean', ascending=False)
summary_score_df.groupby('model_architecture')['auc'].mean()\
    .reset_index(name='auc_mean')\
    .sort_values('auc_mean', ascending=False)

### summary_score_df

In [ ]:
summary_score_df.groupby('feature_set')['auc'].mean()\
    .reset_index(name='auc_mean')\
    .sort_values('auc_mean', ascending=False)

In [ ]:
summary_score_df\
    .groupby(['model_architecture','feature_set','observation'])['auc']\
    .mean()\
    .reset_index(name='auc_mean')\
    .sort_values('auc_mean', ascending=False)

In [ ]:
best_observation=summary_score_df.groupby('observation')['auc']\
    .mean().to_frame().reset_index()\
    .sort_values('auc',ascending=False)\
    .iloc[0,0]
best_observation

### summary_score_df2

In [ ]:
summary_score_df2=summary_score_df\
    .groupby(['model_architecture','feature_set','observation'])['auc']\
    .mean()\
    .reset_index(name='auc_mean')\
    .sort_values('auc_mean', ascending=False)

### summary_score_df3

In [ ]:
summary_score_df3=pd.merge(
    summary_score_df2.groupby('observation')['auc_mean'].max().to_frame().reset_index(),
    summary_score_df2
    , how = 'left'
    , on = ['observation','auc_mean']
).astype({'observation':int}).sort_values('observation',ascending=True)

summary_score_df3

### _selected_hp

In [ ]:
_selected_hp=summary_score_df3\
    .apply( lambda x: f"{x['model_architecture']}_{x['feature_set']}_{x['observation']}", axis=1)\
    .values.tolist()

_selected_hp

## Final Model

### สร้าง hyper_params เฉพาะ ที่จะเอา

In [ ]:
# Hypyer params แต่ละ observation ที่ให้ AUC สูงสุด
_selected_hp=[
    # 'lstm_SAP_6',
    # 'lstm_APACHE_6',
    # 'lstm_SAP_12',    
    # 'lstm_SAP_24',
    # 'rnn_SAP_48',
    # 'gru_SAP_48',
    # 'lstm_SAP_72',
    # 'rnn_SAP_48',
    'rnn_SAP_72',
]

# print(hyper_params['lstm_SAP_6'])

In [ ]:
# เฉพาะใน _selected_hp
for hpk in _selected_hp:
    hyper_params_key=hpk
    model_arch = hyper_params[hyper_params_key]['model_arch']    
    model=eval(f"model_{model_arch}(hyper_params[hyper_params_key],debug=True)")     
    print(hpk, model_arch)

    logdir = os.path.join(f"{MODEL}", f"{model._name}")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
    print('Tensorboard')

    history=model.fit(            
        x=[ get_datascale(hyper_params[hyper_params_key],i).tolist() 
               for i in _x_train_all_ts_df.data_scale],
        y=_y_train_all_df.y.values.tolist(),
        batch_size=100,
        verbose=0,
        epochs=100,
        class_weight=class_weight,
        callbacks=[tensorboard_callback],
        validation_data=[[ get_datascale(hyper_params[hyper_params_key],i).tolist() for i in _x_val_ts_df.data_scale],_y_val_df.y.values.tolist()]
    )
    model.save(f'model/{hyper_params_key}.h5')

In [ ]:
import re
from sklearn import metrics

all_predict_fold={}
predict_fold={}    
for hpk in _selected_hp:
    _model=tf.keras.models.load_model(f'model/{hpk}.h5')
    _hyper_params_key=hpk
    _prediction_result=_model.predict(
            [ get_datascale(hyper_params[hpk],i).tolist() 
             for i in _x_predict_ts_df.data_scale
            ]
        ).flatten().tolist()
    fpr, tpr, thresholds = metrics.roc_curve(_y_predict_df.y.values.tolist(),_prediction_result, pos_label=1)
    roc_auc=metrics.auc(fpr, tpr)
    print(_hyper_params_key)
    predict_fold[_hyper_params_key]={
        'fpr':fpr
        ,'tpr':tpr
        ,'thresholds': thresholds
        ,'roc_auc':roc_auc}
    
def get_auc(item):
    auc=item[1]['roc_auc']
    return auc
sorted_auc=dict(sorted(predict_fold.items(), key=get_auc,reverse=True) )
all_predict_fold[list(sorted_auc.keys())[0]]=sorted_auc[list(sorted_auc.keys())[0]]


plt.figure(dpi=100)
for k in sorted_auc.keys():
    # print(sorted_auc[k]['roc_auc'])
    fpr=sorted_auc[k]['fpr']
    tpr=sorted_auc[k]['tpr']
    roc_auc=sorted_auc[k]['roc_auc']    
    plt.plot(fpr,tpr,label=f"{k}, AUC={roc_auc:.2f}")

plt.legend()
plt.savefig(f'fig/selected_hyperparams.jpg')
plt.show()            

In [ ]:
best_model_key = list(dict(sorted(all_predict_fold.items(), key=get_auc,reverse=True)).keys())[0]

print(best_model_key, f"{all_predict_fold[best_model_key]['roc_auc']:.4f}")

In [ ]:
from sklearn.metrics import roc_auc_score
from math import sqrt

# def roc_auc_ci(y_true, y_score, positive=1):
#     AUC = roc_auc_score(y_true, y_score)
# N1=No.of Positive
# N2=No.of Negative
params={
    'AUC': 0,
    'N1':0,
    'N2':0,
}
def roc_auc_ci(params=params, positive=1):
    AUC = params['AUC']
    N1 = params['N1']
    N2 = params['N2']
    Q1 = AUC / (2 - AUC)
    Q2 = 2*AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC*(1 - AUC) + (N1 - 1)*(Q1 - AUC**2) + (N2 - 1)*(Q2 - AUC**2)) / (N1*N2))
    lower = AUC - 1.96*SE_AUC
    upper = AUC + 1.96*SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (f"({lower:.2f}-{upper:.2f})")

In [ ]:
_y_predict_df.groupby(['y']).y.count().reset_index(name='count')

In [ ]:
params={
    'AUC': all_predict_fold[best_model_key]['roc_auc'],
    'N1':80,
    'N2':1355,
}
roc_auc_ci(params=params, positive=1)

In [ ]:
dict(sorted(all_predict_fold.items(), key=get_auc,reverse=True)).keys()

In [ ]:
from sklearn import metrics
fpr=all_predict_fold[best_model_key]['fpr']
tpr=all_predict_fold[best_model_key]['tpr']
thresholds=all_predict_fold[best_model_key]['thresholds']



roc_auc=metrics.auc(fpr, tpr)
# calculate the g-mean for each threshold
gmeans = np.sqrt(tpr * (1-fpr))
J=tpr-fpr
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
ixJ= np.argmax(J)

In [ ]:
roc_auc,f"{thresholds[ixJ]:.2f}",ix,ixJ

In [ ]:
plt.figure(dpi=100)
lw = 2
plt.plot(
    fpr,
    tpr,
    color="darkorange",
    lw=lw,
    label="ROC curve (area = %0.2f, thresholds=%0.2f)" % (roc_auc,thresholds[ixJ] ),
)
plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title(f"AUROC {best_model_key}")
plt.title(f"AUROC of RNN with SAP-II at 72 hours")
plt.legend(loc="lower right")
plt.show()

In [ ]:
def get_datascale(param, data_scale):
    result=np.vstack(
        
            [
                data_scale[0:param['observation'],j] # rows
                for j in [ts_columns.index(i) for i in eval(f"{param['feature_set']}_columns")] # cols
            ]
        
        ).transpose()
    return result

In [ ]:
print(hyper_params[best_model_key])
print(SAP_columns)
model=tf.keras.models.load_model(f'model/{best_model_key}.h5')
final_prediction_result=model.predict([ get_datascale(hyper_params[best_model_key],i).tolist() 
                                           for i in _x_predict_ts_df.data_scale])


In [ ]:
y_p = [ 0 if i <=thresholds[ixJ] else 1 for i in final_prediction_result ]

In [ ]:
l_yp=len(y_p)
correct_0=0
correct_1=0
for i in range(l_yp):
    if y_p[i]==_y_predict_df.y.values.tolist()[i]:
        if y_p[i]==0:
            correct_0=correct_0+1
        else:
            correct_1=correct_1+1
print(correct_0, correct_1)

In [ ]:
cm = confusion_matrix(_y_predict_df.y.values.tolist(),y_p)
cm

In [ ]:
cm = confusion_matrix(y_p,_y_predict_df.y.values.tolist())
cm

In [ ]:
ax=plot_confusion_matrix(_y_predict_df.y.values.tolist()
                      ,y_p
                      ,classes=(1,0)
                      , normalize=True, title=f"Confusion matrix of the final model (nomalized)\n")

In [ ]:
model.save('final_model.h5')

In [ ]:
# Save Parameters
# best_model_key='gru_SAP_48'

all_params={
    'best_model_key': best_model_key,
    'hyper_params':hyper_params,
    '_x_val_ts_df':_x_val_ts_df,
    '_x_train_ts_df':_x_train_ts_df,
    '_x_train_all_ts_df':_x_train_all_ts_df,
    '_x_predict_ts_df': _x_predict_ts_df,
    '_y_train_all_df':_y_train_all_df,
    '_y_train_df':_y_train_df,
    '_y_val_df':_y_val_df,
    '_y_predict_df':_y_predict_df,
}
with open('./SHAP/all_params.pickle','wb') as handle:
    pickle.dump(all_params,handle)

# !!!! DONE !!!!!!!

# Go to SHAP.ipynb